# Data Gathering and Management Lab - 094290
## Lab 3 - Data Integration
#### Nitzan Shamir 206348187 & Omer Shubi 312236219

In [2]:
### Imports ###,


from sklearn.neighbors import KNeighborsClassifier

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SQLContext
from scipy.stats import beta 
from sklearn import preprocessing
import os
from pyspark.ml.feature import Normalizer

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.elasticsearch:elasticsearch-hadoop:7.9.3 pyspark-shell'

!python -m pip install elasticsearch
from elasticsearch import Elasticsearch

from pyspark.sql.functions import date_format

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd # for small table fancy display only

import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.linalg import SparseVector, DenseVector,Vectors, VectorUDT
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import RegressionEvaluator

from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.pyplot as plt 
import numpy as np

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.mllib.evaluation import MulticlassMetrics

!pip install opencage
import sys
from opencage.geocoder import OpenCageGeocode

import matplotlib.pyplot as plt

### Constants ###

# ES_HOST = 'da2020w-0001.eastus.cloudapp.azure.com'
ES_HOST = '10.0.0.5'
es = Elasticsearch([{'host': ES_HOST}], timeout=600000)

elastic_settings_mappings_basic = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    }
}

elastic_settings_mappings_centorids = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    },
    "mappings": {
        "properties": {
            "centroids" : { "type": "geo_point" }
        }
    }
}

elastic_settings_mappings_date = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    },
    "mappings": {
        "properties": {
            "date" : { "type": "date"},
        }
    }
}

width = 1000
lower_thresh = 0
upper_thresh = 600
grouped_thresh = 0.01

min_date = '2017-07-03'
max_date = '2018-09-11'

### Helper Functions ###
@F.udf
def sample_beta(prev_val, uncertainPercentage, a, b, scale, loc, random_state):
  if uncertainPercentage > grouped_thresh:
    return float(beta.rvs(a=a ,b=b ,scale=scale, loc=loc, random_state=random_state))
  else:
    return float(prev_val)
  
@F.udf
def checkLocation(areaId, locationList):
  if areaId in locationList:
    return areaId
  else:
    return None
  
@F.udf
def checkDate(date, event_dates_list):
  if date in event_dates_list:
    return 1
  else:
    return 0  

@F.udf
def hour_group(hour):
  if hour < 4:
    return int(0) # '0-3'
  elif hour < 8:
    return int(1) # '4-7'
  elif hour < 12:
    return int(2) # '8-11'
  elif hour < 16:
    return int(3) #'12-15'
  elif hour < 20:
    return int(4) #'16-19'
  else:
    return int(5) #'20-23'
  
@F.udf
def getdate(date):
  date = date.split(" ")
  month = {'January': '01',
    'February': '02',
   'March': '03',
  'April': '04',
   'May': '05',
  'June': '06',
   'July': '07',
   'August': '08',
   'September': '09',
  'October': '10',
  'November': '11',
  'December': '12'}[date[1]]
  return f'{date[2]}-{month}-{date[0]}'
  

def column_to_index(c, df):
  indexer = StringIndexer(inputCol=c, outputCol= c + "Index")
  return indexer.fit(df).transform(df)

def column_to_one_hot(c, indexed_df):
  encoder = OneHotEncoderEstimator(inputCols=[c],outputCols=[c + "Vec"])
  return encoder.fit(indexed_df).transform(indexed_df)

def sparse_to_array(v):
  v = DenseVector(v)
  new_array = list([float(x) for x in v])
  return new_array

sparse_to_array_udf = F.udf(sparse_to_array, ArrayType(FloatType()))

to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

def write_to_elastic(df_to_upload, index_name: str, settings):
  
    if not es.indices.exists(index_name): # and overwrite:
        #es.indices.delete(index=index_name)
        es.indices.create(index=index_name, ignore=400, body=settings)

        df_to_upload.write.format("org.elasticsearch.spark.sql")\
            .option("es.resource", index_name)\
            .option("es.nodes.wan.only","true")\
            .option("es.port","9200")\
            .option("es.nodes",ES_HOST)\
            .option("es.nodes.client.only", "false")\
            .save()
    else:
        print("Index already exists")
        
def read_elastic(index, query="", scroll_size="10000", array_field=""):
    if not es.indices.exists(index):
        raise Exception("Index doesn't exist")

    return spark.read\
                .format("org.elasticsearch.spark.sql")\
                .option("es.nodes.wan.only","true")\
                .option("es.port","9200")\
                .option("es.nodes",ES_HOST)\
                .option("es.nodes.client.only", "false")\
                .option("pushdown", "true")\
                .option("es.query", query)\
                .option("es.scroll.size", scroll_size)\
                .option("es.scroll.keepalive", "120m")\
                .option("es.read.field.as.array.include", array_field)\
                .load(index)

  

# Create Aggregate data for prediction Task
def aggregate_data(data_frame):
  processed_df = data_frame.withColumn('hourGroup', hour_group(F.col('currentHour')).cast("int")).withColumn('day', date_format("date", 'yyyy-MM-dd'))

  ### Select Relevant Columns ### 
  processed_df = processed_df.select("day", "areaId1", "distanceCovered", "ellapsedTime", "journeyPatternId", "vehicleId", 'busStop', "hourGroup", "congestion", "locationAread1", "eventDate")

  ### OneHot Encode areadId1 ### 
  indexed1 = column_to_index("areaId1",processed_df).drop("areaId1")
  encoded1 = column_to_one_hot("areaId1Index", indexed1)\
             .withColumn('areaIds', sparse_to_array_udf('areaId1IndexVec')).drop("areaId1IndexVec")


  num_area_id = len(encoded1.select("areaIds").head(1)[0][0])

  ### Groupby & Aggregate ###
  grouped_df = encoded1.groupBy("journeyPatternId","vehicleId", "day", "hourGroup")\
                       .agg(F.array(*[F.max(F.col("areaIds")[i]) for i in range(num_area_id)]).alias("areaIds"),\
                            F.sum("distanceCovered").alias("TotalDistance"), F.sum("ellapsedTime").alias("TotalTime"),\
                            F.countDistinct("busStop").alias("NumberOfBusStop"),\
                            (F.sum(F.col("congestion").cast("int"))/F.count("congestion")).alias("congestionPercentage"),\
                             F.countDistinct("locationAread1").alias("NumberOfLocations"),\
                            F.max('eventDate').alias("isEventDate"),\
                           F.count("*").alias("recordCount"))\
                        .withColumn("isWeekend", date_format("day", 'EEE').isin(["Sat", "Sun"]).cast("int"))\
                        .drop("day", "vehicleId")
  return grouped_df
 
  
# Process data for prediction task  - 1. OneHot Encode journeyPatternId & hourGroup; 2. Assemble as Vector  
def encode_and_assemble(data_frame, dummy=False):
  ### OneHot Encode journeyPatternId & hourGroup ###
  data_frame_fixed = data_frame.withColumn('isEventDate', F.col('isEventDate').cast('int'))
  indexed2 = column_to_index("journeyPatternId",data_frame_fixed)
  encoded2 = column_to_one_hot("journeyPatternIdIndex", indexed2)\
              .withColumn('journeyPatternIds', sparse_to_array_udf('journeyPatternIdIndexVec'))\
              .drop("journeyPatternIdIndexVec","journeyPatternIdIndex")

  encoded2 = column_to_index("hourGroup",encoded2)
  encoded2 = column_to_one_hot("hourGroupIndex", encoded2)\
              .withColumn('hourGroups', sparse_to_array_udf('hourGroupIndexVec'))\
              .drop("hourGroupIndexVec", "hourGroupIndex")
  encoded3 = encoded2.withColumn("areaIds", to_vector("areaIds"))\
                     .withColumn("journeyPatternIds", to_vector("journeyPatternIds"))\
                     .withColumn("hourGroups", to_vector("hourGroups"))
  
  features_list = ["areaIds", "TotalDistance", "NumberOfBusStop", "isWeekend", "journeyPatternIds", "hourGroups", "TotalTime", "NumberOfLocations", 'isEventDate']
  if dummy:
    features_list.append("isTripUncertain")
  assembler = VectorAssembler(
      inputCols=features_list,
      outputCol="features")

  input_data = assembler.transform(encoded3)
  
  input_data = input_data.select("*",F.when(F.col("congestionPercentage") > 0, 1.0).otherwise(0.0).alias('isCongestion'))
  
  return input_data


def recreate_warehouse_df(USE = "take"):
  # USE = "take" / 'sample' / 'full'
  ### Schema ###
  full_schema = StructType([StructField('_id',StructType([StructField('$oid',StringType(),True)]),True),
                       StructField('actualDelay',LongType(),True),
                       StructField('angle',DoubleType(),True),
                       StructField('anomaly',BooleanType(),True),
                       StructField('areaId',LongType(),True),
                       StructField('areaId1',LongType(),True),
                       StructField('areaId2',LongType(),True),
                       StructField('areaId3',LongType(),True),
                       StructField('atStop',BooleanType(),True),
                       StructField('busStop',LongType(),True),
                       StructField('calendar',StructType([StructField('$numberLong',StringType(),True)]),True),
                       StructField('congestion',BooleanType(),True),
                       StructField('currentHour',LongType(),True),
                       StructField('dateType',LongType(),True),
                       StructField('dateTypeEnum',StringType(),True),
                       StructField('delay',LongType(),True),
                       StructField('direction',LongType(),True),
                       StructField('distanceCovered',DoubleType(),True),
                       StructField('ellapsedTime',LongType(),True),
                       StructField('filteredActualDelay',LongType(),True),
                       StructField('gridID',StringType(),True),
                       StructField('journeyPatternId',StringType(),True),
                       StructField('justLeftStop',BooleanType(),True),
                       StructField('justStopped',BooleanType(),True),
                       StructField('latitude',DoubleType(),True),
                       StructField('lineId',StringType(),True),
                       StructField('loc',StructType([StructField('coordinates',ArrayType(DoubleType(),True),True),StructField('type',StringType(),True)]),True),
                       StructField('longitude',DoubleType(),True),
                       StructField('poiId',LongType(),True),
                       StructField('poiId2',LongType(),True),
                       StructField('probability',DoubleType(),True),
                       StructField('systemTimestamp',DoubleType(),True),
                       StructField('timestamp',StructType([StructField('$numberLong',StringType(),True)]),True),
                       StructField('vehicleId',LongType(),True),
                       StructField('vehicleSpeed',LongType(),True)])  


  ### Load and Transfrom df ###
  df = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile', schema=full_schema)
  if USE=='take':
    df = spark.createDataFrame(df.take(10000)).cache()
  elif USE =='sample':
    df = df.sample(False, 0.001, seed=0).cache()
  elif USE=='full':
    pass

  df = df.withColumn('timestamp', df['timestamp']['$numberLong'].cast('bigint'))\
         .withColumn('calendar', df['calendar']['$numberLong'].cast('bigint'))

  df = df.withColumn('date', F.from_unixtime((df['timestamp'])/1000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
         .withColumn('loc', df['loc']['coordinates'])\
         .drop("timestamp")\
         .withColumn('calendar', F.from_unixtime((df['calendar'])/1000000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
         .withColumn('systemTimestamp', F.to_timestamp(df['systemTimestamp']))


  df = df.drop("_id","calendar", "systemTimestamp", "probability", "poiId", "poiId2", "longitude", "latitude", "direction", "dateTypeEnum", "dateType", "currentHour", "angle", "filteredActualDelay", 'anomaly', 'gridID')

  df = df.withColumn("is_weekend", date_format("date", 'EEE').isin(["Sat", "Sun"]).cast("int"))

  df = df.withColumn("currentHour", F.hour("date"))

  df = df.withColumn('ellapsedTime',df['ellapsedTime']/1000)
  
  df = df.drop('lineId')

  df = df.dropDuplicates()

  return df


# Load from Elastic
def load_data():
#   ES_HOST = 'da2020w-0001.eastus.cloudapp.azure.com'
  ES_HOST = "10.0.0.5"

  es = Elasticsearch([{'host': ES_HOST}], timeout=600000)
  df = read_elastic('dublin-buses-main-df')
  areaId_df = read_elastic('dublin-buses-area-id')
  df = df.join(areaId_df, on='areaId')
  return df


def evaluate_model(predictions, name):
  predictionAndTarget = predictions.select("prediction", "isCongestion")
  
  metrics = MulticlassMetrics(predictionAndTarget.rdd)
   
  # Get metrics
  acc = metrics.accuracy
  Recall= metrics.recall(1)

  print(f"Accuracy {name}: {acc}")
  print(f"Recall {name}: {Recall}")

def regression_task(data, maxIter, train_ratio, USE='full'):

  # regression Model
  lr = LogisticRegression(featuresCol='features', labelCol='isCongestion').setMaxIter(maxIter)
  
  if USE == 'full':
    train, test = data.randomSplit([train_ratio, 1-train_ratio], seed=12345)
  else: # USE=='take'
    train, test = (spark.createDataFrame(data.take(1000))).randomSplit([train_ratio, 1-train_ratio], seed=12345)

  # Fit and Predict 
  lrModel = lr.fit(train)
  predictions_test = lrModel.transform(test)
  predictions_train = lrModel.transform(train)
  
  # Create both evaluators
  evaluatorMulti = MulticlassClassificationEvaluator(labelCol="isCongestion", predictionCol="prediction")
  
  evaluate_model(predictions_train, "Train")
  evaluate_model(predictions_test, "Test")

def KNN(XTrain, yTrain, XTest):
  knn = KNeighborsClassifier(n_neighbors=1)
  knn.fit(XTrain, yTrain)
  predictions = knn.predict(XTest)
  return predictions



def get_locations_from_addresses(addresses_file='/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/addresses_txt-1.txt'):
  key = '0fddcc5e021f474883fb249ebd21cfc3'
  geocoder = OpenCageGeocode(key)
  locations = []
  try: 
    with open(addresses_file, 'r') as f:
      for line in f:
        address = line.strip()+ ", Dublin, Ireland"
        results = geocoder.geocode(address, no_annotations='1')

        if results and len(results):
          longitude = results[0]['geometry']['lng']
          latitude  = results[0]['geometry']['lat']
        else:
          sys.stderr.write("not found: %s\n" % address)
        
        locations.append([address, longitude, latitude])

  except RateLimitExceededError as ex:
    print(ex)

  return spark.createDataFrame(locations,schema=['locationName','longitude','latitude'])

In [3]:
%sh pip install lxml

Collecting lxml
 Downloading https://files.pythonhosted.org/packages/d2/88/b25778f17e5320c1c58f8c5060fb5b037288e162bd7554c30799e9ea90db/lxml-4.6.2-cp37-cp37m-manylinux1_x86_64.whl (5.5MB)
Installing collected packages: lxml
Successfully installed lxml-4.6.2
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

# Part 1 & Part 2 Overview:

In part 1 & part 2 we aim to improve our **Trip Congestion** model from task 2 by **hydrating** the model with more information.

Recall that in task 2 we  **Predict whether a bus trip includes congestion.**

We define 'bus trip' as a trip taken by a bus on a specific trip, from start to finish. 

Specifically, each trip is defined by the following features:

 * Numerical features:
   * Number of bus stops passed through in trip 
   * Total distance covered
   * Total trip time
 * Categorial features:
   * Whether the trip was during the weekend or not
   * Area id (all areas passed through)
   * Journy pattern id
   * Hour group
 * Label:
   * isCongestion
   
Partitioning the data into trips is done by aggregating (groupBy) over JournyPatternId, vehicleId, date and hourGroup.

The model we use is a **Logistic Regression** model, and the metrics we focus on are **Accuracy** and **Recall**.

# Part 1 -  Additional Data Source Integration 

In this part we enrich the data with information about **popular locations**, and **wide-spread events**.

In [6]:
df = recreate_warehouse_df(USE = "full")

## Data Source 1 - Popular Locations

It seems reasonable that buses that pass through popular locations have a higher chance of being in congestion.

To get the data we first extract a list of names of popular places from [tourist attractions in Ireland](https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Ireland#D).

Then, we use **[OpenCage](https://opencagedata.com/) Geocoding API** which is an API to convert coordinates to and from places, to perform **forward geocoding**.

For example: `National Museum of Ireland, Dublin, Ireland --> latitude: 53.339881, longitude: -6.253348`

In [8]:
locations_df = get_locations_from_addresses()

In [9]:
write_to_elastic(locations_df, "locations", elastic_settings_mappings_centorids)

### Analyze The Data

In [11]:
display(locations_df)

locationName,longitude,latitude
"Christ Church Cathedral, Dublin, Ireland",-6.2712269,53.3434153
"Chester Beatty Library, Dublin, Ireland",-6.2673618,53.3420435
"Croke Park, Dublin, Ireland",-6.2526326,53.3606275
"Dublin Castle, Dublin, Ireland",-6.2670509,53.342597
"Dublinia, Dublin, Ireland",-6.2720159,53.3432376
"EPIC The Irish Emigration Museum, Dublin, Ireland",-6.2479785,53.3482653
"General Post Office building, Dublin, Ireland",-6.24889,53.33306
"Glasnevin Cemetery, Dublin, Ireland",-6.2785829,53.3720816
"Grafton Street, Dublin, Ireland",-6.2592087,53.3439527
"Ha'penny Bridge, Dublin, Ireland",-6.26311,53.3463205


In [12]:
Image(url="https://drive.google.com/uc?id=1TvdCxWgYdm56vHZajBPypjTb0F2mybF_",  width=650)

From the table above, we can see than we have **29** popular locations that range in type from  museums, stadiums and bars.

In figure 1, we look at the popular locations located on the map. From it, we can see that most of the popular locations are located in the Dublin's center, but there are also few farther away, such as the airport.

## Data Source 2 - Wide-Spread Events

It is known that major events that happen around the city can impact the traffic condition.

Therefore, we aim to relate between dates of events that happened around the city and bus trips.

We focus on two major hubs - 

* **[Music Concerts at the 3Arena amphitheatre](https://en.wikipedia.org/wiki/3Arena)**
* **[Football, Rugby and concerts at Aviva Stadium](https://en.wikipedia.org/wiki/Aviva_Stadium)** 

The web scraping process includes:
1. Detect and extract tables that store relevant information from the two websites.
2. Drop erronous records.
3. Parse the 'Dates' to be machine-readable.
4. Merge the different tables containing all events based on the event type and date.

In [15]:
website_url = 'https://en.wikipedia.org/wiki/List_of_events_at_the_3Arena'
List_of_events_at_the_3Arena = pd.read_html(website_url)
events_at_3arena = spark.createDataFrame(List_of_events_at_the_3Arena[1])
events_at_3arena = events_at_3arena.where(~F.col("Dates").contains('and'))\
                                   .where(~F.col("Dates").contains('['))\
                                   .withColumn('parsedDates',F.to_date(getdate(F.col("Dates"))))\
                                   .withColumn('eventType', F.lit('Music Concerts at 3Arena'))

display(events_at_3arena)

Event/Artist/Band,Dates,Tour / Notes,parsedDates,eventType
The Pussycat Dolls,1 February 2009,Doll Domination Tour,2009-02-01,Music Concerts at 3Arena
AC/DC,18 April 2009,Black Ice World Tour,2009-04-18,Music Concerts at 3Arena
Def Leppard,12 June 2009,With Whitesnake & Journey[1],2009-06-12,Music Concerts at 3Arena
Green Day,21 October 2009,21st Century Breakdown World Tour,2009-10-21,Music Concerts at 3Arena
Muse,6 November 2009,The Resistance Tour,2009-11-06,Music Concerts at 3Arena
Backstreet Boys,13 November 2009,This Is Us Tour,2009-11-13,Music Concerts at 3Arena
Arctic Monkeys,26 November 2009,Humbug Tour,2009-11-26,Music Concerts at 3Arena
Lily Allen,8 December 2009,"It's Not Me, It's You World Tour",2009-12-08,Music Concerts at 3Arena
Depeche Mode,10 December 2009,Tour of the Universe,2009-12-10,Music Concerts at 3Arena
Thirty Seconds to Mars,26 February 2010,Into the Wild Tour,2010-02-26,Music Concerts at 3Arena


In [16]:
website_url = 'https://en.wikipedia.org/wiki/Aviva_Stadium'
Aviva_Stadium_df = pd.read_html(website_url)

In [17]:
rugby_matches_at_Aviva_Stadium = Aviva_Stadium_df[1]
rugby_matches_at_Aviva_Stadium = spark.createDataFrame(rugby_matches_at_Aviva_Stadium)
rugby_matches_at_Aviva_Stadium = rugby_matches_at_Aviva_Stadium.where(~F.col("('Rugby union matches', 'Date')").contains("Row"))\
                                   .withColumn('parsedDates',F.to_date(getdate(F.col("('Rugby union matches', 'Date')"))))\
                                   .withColumn('eventType', F.lit('Rugby Matches at Aviva Stadium'))
display(rugby_matches_at_Aviva_Stadium)

"('Rugby union matches', 'Date')","('Rugby union matches', 'Competition')","('Rugby union matches', 'Home')","('Rugby union matches', 'Away')","('Rugby union matches', 'Score')","('Rugby union matches', 'Attendance')",parsedDates,eventType
31 July 2010,Challenge match,Leinster/Ulster,Munster/Connacht,68–0,34000,2010-07-31,Rugby Matches at Aviva Stadium
2 October 2010,2010-11 Magners League,Leinster,Munster,13–9,50645,2010-10-02,Rugby Matches at Aviva Stadium
6 November 2010,Test match,Ireland,South Africa,21–23,35515,2010-11-06,Rugby Matches at Aviva Stadium
13 November 2010,Test match,Ireland,Samoa,20–10,37955,2010-11-13,Rugby Matches at Aviva Stadium
20 November 2010,Test match,Ireland,New Zealand,18–38,49640,2010-11-20,Rugby Matches at Aviva Stadium
28 November 2010,Test match,Ireland,Argentina,29–9,45340,2010-11-28,Rugby Matches at Aviva Stadium
18 December 2010,2010–11 Heineken Cup pool stage,Leinster,Clermont Auvergne,24–8,44873,2010-12-18,Rugby Matches at Aviva Stadium
13 February 2011,2011 Six Nations Championship,Ireland,France,22–25,51700,2011-02-13,Rugby Matches at Aviva Stadium
19 March 2011,2011 Six Nations Championship,Ireland,England,24–8,51700,2011-03-19,Rugby Matches at Aviva Stadium
9 April 2011,2010–11 Heineken Cup quarter-final,Leinster,Leicester Tigers,17–10,49672,2011-04-09,Rugby Matches at Aviva Stadium


In [18]:
football_matches_at_Aviva_Stadium = Aviva_Stadium_df[4]
football_matches_at_Aviva_Stadium = spark.createDataFrame(football_matches_at_Aviva_Stadium)
football_matches_at_Aviva_Stadium = football_matches_at_Aviva_Stadium.withColumn('parsedDates',F.to_date(getdate(F.col("('Football matches', 'Date')"))))\
                                                                     .withColumn('eventType', F.lit('Football Matches at Aviva Stadium'))
display(football_matches_at_Aviva_Stadium)

"('Football matches', 'Date')","('Football matches', 'Competition')","('Football matches', 'Home')","('Football matches', 'Away')","('Football matches', 'Score')","('Football matches', 'Attendance')",parsedDates,eventType
31 July 2010,Challenge Match,League of Ireland XI,Manchester United,1–7,49800,2010-07-31,Football matches at Aviva Stadium
11 August 2010,Friendly,Republic of Ireland,Argentina,0–1,45133,2010-08-11,Football matches at Aviva Stadium
7 September 2010,Euro 2012 qualification,Republic of Ireland,Andorra,3–1,40283,2010-09-07,Football matches at Aviva Stadium
8 October 2010,Euro 2012 qualification,Republic of Ireland,Russia,2–3,50411,2010-10-08,Football matches at Aviva Stadium
14 November 2010,2010 FAI Cup Final,Sligo Rovers,Shamrock Rovers,0–0 (aet) (2–0 pens),36101,2010-11-14,Football matches at Aviva Stadium
17 November 2010,Friendly,Republic of Ireland,Norway,1–2,25000,2010-11-17,Football matches at Aviva Stadium
8 February 2011,2011 Nations Cup,Republic of Ireland,Wales,3–0,19783,2011-02-08,Football matches at Aviva Stadium
9 February 2011,2011 Nations Cup,Scotland,Northern Ireland,3–0,18742,2011-02-09,Football matches at Aviva Stadium
26 March 2011,Euro 2012 qualification,Republic of Ireland,North Macedonia,2–1,32000,2011-03-26,Football matches at Aviva Stadium
29 March 2011,Friendly,Republic of Ireland,Uruguay,2–3,15000,2011-03-29,Football matches at Aviva Stadium


In [19]:
concerts_at_Aviva_Stadium = Aviva_Stadium_df[6]
concerts_at_Aviva_Stadium = spark.createDataFrame(concerts_at_Aviva_Stadium)
concerts_at_Aviva_Stadium = concerts_at_Aviva_Stadium.withColumn('parsedDates',F.to_date(getdate(F.col("('Concerts at the Aviva Stadium', 'Date')"))))\
                                                     .withColumn('eventType', F.lit('Concerts at Aviva Stadium'))

display(concerts_at_Aviva_Stadium)

"('Concerts at the Aviva Stadium', 'Date')","('Concerts at the Aviva Stadium', 'Artist')","('Concerts at the Aviva Stadium', 'Tour')","('Concerts at the Aviva Stadium', 'Attendance')",parsedDates,eventType
24–25 September 2010,Michael Bublé,Crazy Love Tour,"100,000[110]",null,Concerts at Aviva Stadium
25 June 2011,Neil Diamond,World Tour 2011,50108,2011-06-25,Concerts at Aviva Stadium
2 July 2011,The Script,Science & Faith Tour,47910,2011-07-02,Concerts at Aviva Stadium
24 July 2012,Madonna,The MDNA Tour,33953,2012-07-24,Concerts at Aviva Stadium
15 September 2012,Lady Gaga,The Born This Way Ball,37005,2012-09-15,Concerts at Aviva Stadium
14 June 2013,Robbie Williams,Take the Crown Stadium Tour,"65,000[111]",2013-06-14,Concerts at Aviva Stadium
21 June 2013,Rihanna,Diamonds World Tour,48482,2013-06-21,Concerts at Aviva Stadium
18 September 2013,Roger Waters,The Wall Live,24210,2013-09-18,Concerts at Aviva Stadium
1 July 2015,AC/DC,Rock or Bust World Tour,"50,000[112]",2015-07-01,Concerts at Aviva Stadium
21 June 2016,Rihanna,Anti World Tour,29017,2016-06-21,Concerts at Aviva Stadium


In [20]:
events_types_dates = events_at_3arena.select("eventType", "parsedDates").union(rugby_matches_at_Aviva_Stadium.select("eventType", "parsedDates"))\
                                                    .union(football_matches_at_Aviva_Stadium.select("eventType", "parsedDates"))\
                                                    .union(concerts_at_Aviva_Stadium.select("eventType", "parsedDates"))
write_to_elastic(events_types_dates.withColumnRenamed('parsedDates','date'), "events-types-dates", elastic_settings_mappings_date)

In [21]:
events_types_dates = read_elastic('events-types-dates')
events_types_dates = events_types_dates.dropna()

### Analyze The Data

In [23]:
display(events_types_dates)

date,eventType
2016-04-05T00:00:00.000+0000,Music Concerts at 3Arena
2016-12-08T00:00:00.000+0000,Music Concerts at 3Arena
2017-01-20T00:00:00.000+0000,Music Concerts at 3Arena
2017-04-03T00:00:00.000+0000,Music Concerts at 3Arena
2017-05-06T00:00:00.000+0000,Music Concerts at 3Arena
2017-05-11T00:00:00.000+0000,Music Concerts at 3Arena
2017-05-20T00:00:00.000+0000,Music Concerts at 3Arena
2017-05-27T00:00:00.000+0000,Music Concerts at 3Arena
2017-06-14T00:00:00.000+0000,Music Concerts at 3Arena
2017-10-27T00:00:00.000+0000,Music Concerts at 3Arena


In [24]:
display(events_types_dates.groupby("eventType").count())

eventType,count
Music Concerts at 3Arena,105
Concerts at Aviva Stadium,12
Rugby Matches at Aviva Stadium,94
Football matches at Aviva Stadium,100


In [25]:
Image(url="https://drive.google.com/uc?id=1zotQ0Qn43958oc4Tq6WL1CqRIBtSRhjR",  width=width+400)

In [26]:
events_grouped_date = events_types_dates.where(F.col("date").between(min_date,max_date)).groupby("date").count()

x = list(events_grouped_date.select("date").rdd.flatMap(lambda x: x).collect())
y = list(events_grouped_date.select("count").rdd.flatMap(lambda x: x).collect()) 
   
fig = plt.figure(figsize = (20,15)) 
  
# creating the bar plot 
plt.bar(x, y, color ='maroon', width = 3) 
  
plt.xlabel("Date") 
plt.ylabel("Event Count") 
plt.title("Events per Day") 
# plt.xticks(ticks=x, labels=[val.strftime("%m/%d/%Y") for val in x], rotation=90)
plt.show() 

From the tables above, we can see than we have **311** events that range in type from music concerts to different sport matches.

From figure 2, we can see that we have **4** different event types: Music Concerts at the 3Arena amphitheatre, Football matches, Rugby matches and concerts at Aviva Stadium.

There are roughly 100 events of each type, except concerts at Aviva Stadium, which are only 12.

In figure 3 we show a histogram of events per type per month. From it we can see that there are roughly 10 events of each type each month (excluding concerts at Aviva Stadium), each year between 2009 and 2020.

In figure 4 we show a histogram of events per type per date filtered by the bus records range in the dublin buses data. From it we can see that only a few days have more than one event, and that there are many days without any events at all.

Note: we also have more specific information about match scores, and who performed at the concerts, but we do not make use of these.

## Integration Of The Data

**We enrich the bus trips by determining whether or not an event happened on the same day as the bus trip or not, and by informing it by how many areas of popular locations it passed through.**

To **relate between the popular locations and the bus trips**:

1. We use the areaId1 attribute, and we match each popular location to the matching areaId1. We do so by:
  - Compute Centroid of each AreaId1 based on all the raw records
  - Using the **K-Nearest-Neighbours** algorithm with `k=1`, determine which areaId1 is closest to the coordinates of the popular location.
  - In Figure 5 below, we can see the match areaIds of the popular locations. The locations are concetrated in 4 areaIds, in the center of Dublin. 
2. For each raw record, if its areaId1 is the same as the areaId1 of a popular location, we mark it. 
3. For each raw record add a feature 'locationAreaId1', which gets the areaId1 if the areaId1 is an area that is in the popular locations list, and 0 otherwise. 
3. When we aggregate the records into bus trips, we count through how many different such areas the bus passes through.


To **relate between the event to the matching bus trip**:

* We use date attribute, and for each bus trip add a feature 'isEventDate', which gets 1 if the date of the bus trip is a date that is in the events days list, and 0 otherwise.

In [29]:
Image(url="https://drive.google.com/uc?id=1ti0UUf341wGc1TZxaSzPaBQlnNGEIuce",  width=1200)

In [30]:
df_centroids = df.groupby("areaId1").agg(F.mean(df["loc"][0]).alias("lon_mean"), F.mean(df["loc"][1]).alias("lat_mean"))
df_centroids = df_centroids.withColumn("centroids", F.array(F.col("lon_mean"), F.col("lat_mean")))
write_to_elastic(df_centroids, "areasidscentroids", elastic_settings_mappings_centorids)

In [31]:
df_centroids = read_elastic('areasidscentroids')

In [32]:
display(df_centroids)

areaId1,centroids,lat_mean,lon_mean
6,"List(-6.501427299574047, 53.29661658856761)",53.296616,-6.501427
8,"List(-6.383238865649497, 53.302540032351885)",53.30254,-6.383239
17,"List(-6.260894107541568, 53.3652099501078)",53.36521,-6.2608943
13,"List(-6.199013962757952, 53.1934325700584)",53.19343,-6.199014
18,"List(-6.209868221338336, 53.48555613922627)",53.485558,-6.2098684
20,"List(-6.131383165310543, 53.5373057667571)",53.537304,-6.131383
14,"List(-6.266832338406782, 53.3135683773177)",53.31357,-6.2668324
15,"List(-6.105597036523065, 53.166064128859034)",53.166065,-6.105597
16,"List(-6.147341290485041, 53.26560736761332)",53.265606,-6.1473413
9,"List(-6.536128638989444, 53.363134926893515)",53.363136,-6.5361285


In [33]:
locations_df = read_elastic('locations')
locationList = list(locations_df.select("areaId1").distinct().rdd.flatMap(lambda x: x).collect())
print(locationList)

[17, 8, 11, 14]

In [34]:
XTrain = df_centroids.select("lon_mean", "lat_mean").toPandas()
yTrain = df_centroids.select("areaId1").toPandas()
XTest = locations_df.select('longitude','latitude').toPandas()
yPred = KNN(XTrain, yTrain, XTest)
locations_centroids = locations_df.withColumn("centroids", F.array(F.col("longitude"), F.col("latitude")))
locationNames = list(locations_centroids.select("locationName").rdd.flatMap(lambda x: x).collect())
centroids = list(locations_centroids.select("centroids").rdd.flatMap(lambda x: x).collect())
locations_centroids = spark.createDataFrame([[loc, int(area), cent] for loc, area, cent in zip(locationNames, yPred, centroids)], schema=["locationNames", "areaId1", "centroids"])

/local_disk0/tmp/1606665825487-0/PythonShell.py:367: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 :param kwargs: see Magics

In [35]:
display(locations_centroids.select("locationNames", "areaId1"))

locationNames,areaId1
"Christ Church Cathedral, Dublin, Ireland",17
"Chester Beatty Library, Dublin, Ireland",17
"Croke Park, Dublin, Ireland",17
"Dublin Castle, Dublin, Ireland",17
"Dublinia, Dublin, Ireland",17
"EPIC The Irish Emigration Museum, Dublin, Ireland",17
"General Post Office building, Dublin, Ireland",14
"Glasnevin Cemetery, Dublin, Ireland",17
"Grafton Street, Dublin, Ireland",17
"Ha'penny Bridge, Dublin, Ireland",17


In [36]:
## TODO read the big table from elastic and .select("parsedDates")).where(F.col("parsedDates").between(min_date, max_date)).distinct()
event_dates = read_elastic('events').withColumn("date", F.to_date(F.col("date")).cast("string"))
event_dates_list = list(event_dates.select("date").rdd.flatMap(lambda x: x).collect())
print(event_dates_list)

events_dates = events_at_3arena.select("parsedDates").union(rugby_matches_at_Aviva_Stadium.select("parsedDates"))\
                                                    .union(football_matches_at_Aviva_Stadium.select("parsedDates"))\
                                                    .union(concerts_at_Aviva_Stadium.select("parsedDates")).where(F.col("parsedDates").between(min_date, max_date)).distinct()
events_dates = events_dates.withColumnRenamed('parsedDates', 'date')

['2017-12-16', '2018-04-21', '2018-02-24', '2018-02-26', '2018-05-25', '2018-04-01', '2018-04-16', '2018-02-10', '2018-05-10', '2018-05-12', '2018-03-08', '2018-05-26', '2018-05-30', '2018-06-02', '2018-06-23', '2018-08-01', '2018-08-04', '2018-08-20', '2017-11-16', '2017-11-18', '2017-11-25', '2017-08-05', '2017-09-05', '2017-10-06', '2017-10-07', '2017-10-27', '2017-11-05', '2017-11-06', '2017-11-11']

In [37]:
df_hydrated = df.withColumn("eventsList", F.array([F.lit(i) for i in event_dates_list]))\
                .withColumn('day', date_format("date", 'yyyy-MM-dd'))\
                .withColumn("eventDate", checkDate(F.col('day'), F.col('eventsList')))\
                .withColumn("locationList", F.array([F.lit(int(i)) for i in locationList]))\
                .withColumn("locationAread1", checkLocation(F.col('areaId1'), F.col("locationList")))

In [38]:
grouped_df_hydrated = aggregate_data(df_hydrated)
write_to_elastic(grouped_df_hydrated, "datahydrated", elastic_settings_mappings_basic)

### Assumptions needed

When doing the integration proccess we make some assumptions:

1. When relating between the popular locations and the bus trips:
  * To match between a popular location to its areaId, we assume that assigning the areaId whose centroid is the nearest is a good match and that the centroid of all records with a given areaId represent it well.
  * A more qualitive assumption is that bus trips that pass through the same area as the popular location actually pass near that place. We acknowledge that this might be a rough assumption but we still want to check the hypothesis of whether this feature improves the model.
  * When hydrating the locations by using the forward geocoding api, we assumed that the coordinates match those of the actual places. We checked this assumption and saw that the locations match what we see in Google Maps.
  * Additionally, we assumed that the coordinate units (longitude and latitude) of the locations and of our data are the same. We also checked this and it seems to hold.
  
2. When relating between the event to the matching bus trip:
  * We assume that dates of bus trips and dates of events are in the same timezone, so if two dates are the same in the data we assume they are the same in the real world.
  * Additionally, we make a rough qualitive assumption that bus trips that happen on the same day of an event can be impacted by the event, regardless of the specific time.
  Think for example of a football match that starts at 7 PM, already a few hours before the match and up untill late at night it can affect the traffic.

# Part 2 - Task Improvement 

In this part, we aim to improve our **Trip Congestion** model by the hydrated data we created in part 1.

To recap, we added two features - NumberOfLocations (numerical) and isEventDate (boolean) to the data.

The logistic regression is applied in the same way as in task 2, just with the additional features appended.

Note, we use the full data, with the uncertain records, as we aim to compare between the basic model from task 2 to this one.

In [41]:
grouped_df_hydrated = read_elastic('datahydrated',array_field="areaIds")
grouped_df_hydrated = grouped_df_hydrated.where(grouped_df_hydrated['recordCount'] > 20)
display(grouped_df_hydrated)

NumberOfBusStop,NumberOfLocations,TotalDistance,TotalTime,areaIds,congestionPercentage,hourGroup,isEventDate,isWeekend,journeyPatternId,recordCount
32,2,11.416638,4186.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,4,0,0,00010001,107
42,2,17.288464,4738.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,3,0,0,00010001,300
41,2,15.908918,6445.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,2,0,0,00010001,157
42,2,17.671694,4562.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,5,0,0,00010001,175
41,2,12.564413,4724.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,4,0,0,00010001,189
41,2,14.290978,4202.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,4,0,0,00010001,156
42,2,22.853909,6403.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,5,0,0,00010001,272
41,2,18.012093,4686.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,5,0,0,00010001,182
42,2,10.668406,2441.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,4,0,0,00010001,169
42,2,17.270351,3969.0,"List(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,5,0,0,00010001,258


In [42]:
input_data = encode_and_assemble(grouped_df_hydrated)

In [43]:
regression_task(input_data, maxIter = 10, train_ratio = 0.7, USE='full')

Accuracy Train: 0.797715473827051
Recall Train: 0.07165638618134355
Accuracy Test: 0.7957574534631489
Recall Test: 0.07181674180577556

## Result Comparison And Conclusions:

In [45]:
results_info = pd.DataFrame([['data without hydratation', 0.7970904191752517,0.7970539272562175,0.06747116964374761, 0.06784939249414564], 
                              ['hydrate data', 0.797715473827051,0.7957574534631489,0.07165638618134355, 0.07181674180577556]],
                            columns=['Method','Train Accuracy','Test Accuracy', 'Train Recall','Test Recall']).round(5)
results_df = spark.createDataFrame(results_info)
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
data without hydratation,0.79709,0.79705,0.06747,0.06785
hydrate data,0.79772,0.79576,0.07166,0.07182


In [46]:
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
data without hydratation,0.79709,0.79705,0.06747,0.06785
hydrate data,0.79772,0.79576,0.07166,0.07182


In [47]:
display(results_df)

Method,Train Accuracy,Test Accuracy,Train Recall,Test Recall
data without hydratation,0.79709,0.79705,0.06747,0.06785
hydrate data,0.79772,0.79576,0.07166,0.07182


As we see in the table and graphs above, compared to task 2, by hydrating the bus trips with information about days with events and whether or not they passed through a popular location (specifically, how many areas of popular locations), the model accuracy roughly stayed the same, and the model's recall showed a small improvement. (Hover with mouse on bar to see exact values).

Thus we conclude that this hydration has a small positive impact on the model performance and adding more events\locations to the dataset can be of benefit.

# Part 3 - Granularity Level Analysis 

In this part we provide granularity level analyses based on time, location, and bus trips' lengths. Specifically - 

* Average Distance Covered vs Time
* Number of Bus Stops per location (area)
* Average Congestion vs bus trips' lengths

We perform the analyses in Kibana, using the data stored in our datawarehouse, after performing the necessary adaptions to this task.

## Time Analysis

In this part, we present a granularity level analysis of the **average distanced covered**, i.e. we calculate the average distanceCovered of the records by:
- **year** : 2017, 2018.
- **month of the year** : January, February, March, April, May, June, July, August, September, October, November, December.
- **day of the week** : Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday.
- **hour of the day** : 00:00, 01:00, ... , 23:00.

Additionally, we show an analysis of the average distanced covered in general.

We believe that from such an analysis decision makers can get a better understanding about the general traffic congestion and about how crowded the buses are.

For example, if the average distancedCovered at 08:00 AM is much smaller than the average distancedCovered at 14:00 PM, we can conclude that maybe at 08:00 there is more taffic so the bus covers less distance, or maybe, there are more people on the bus so it stayed more time in the bus station to let people get down so it covers less distance, than if the bus is not crowded with people.

And more in general, when looking at the average distancedCovered by week of the day, month of the year and year, similar but more high-level conclusions can be made.

Note: To get reliable and accurate results, we look at constant time frames between the records, so we can conclude on external effects about the distancedCovered. Therfore, we look only on records with ellapsedTime between 18 to 22 seconds. As we already saw in task 2, most of the ellapsedTime values are between 0 to 40, and 20 is the most frequent. After the filtering we are left with ~130M records.

Note: The analysis we present in this part is based on the full data records.

In [51]:
Image(url="https://drive.google.com/uc?id=1NDO532xKGYcGYwMf8DEH73UyEckgBXJ-",  width=1600)

In the dashboard above, we can see histograms of the average 'distancedCovered', by different time granularity levels.

Interesting insights:

1. We see that on average, buses advance 0.104 km on average between two consecutive records.
1. There is barely no difference between 2017 and 2018. However, keep in mind that the data we have is incomplete when looking at years, as it starts from July 2017 and ends in September 2018.
1. When looking at the average distance on a monthly level, there is only a slight difference between months, and the distance covered in september is the lowest. Maybe this is because of the [many exciting events that happen in September?](https://freetoursbyfoot.com/dublin-in-september/)
1. There is a noticeable difference between work days (0.1-0.106) and weekends (0.109-0.112). Interestingly the average distance covered on Monday - the first work day, is higher than the other work days (Tuesday - Friday).
1. The finest granularity provides us with information on an hourly basis. Even though the avg. distance covered is 0.104, we see that during working hours (7-17) the distance covered is smaller than 0.1, and that during the early mornings and late evenings, the avg. distance is considerably higher at 0.101-0.171.
1. There is a dip in the avg. distance covered at 1-2 AM. We relate this to the fact that there are not supposed to be buses travelling at this time, so they are mostly standing still in the bus garage.

## Location Analysis

In this part, we present a granularity level analysis of the **number of bus stops**, i.e. we calculate the number of bus stops by:

- **AreaId1** 
- **AreaId2**
- **AreaId3**
- **AreaId**  

In task 1 we saw that each area feature represents a finer partition of the map, from areaId1 (coarsest granularity level), areaId2 and areaId3 (finer granularity level) up to areaId (finest granularity level).

We believe that from such an analysis decision makers can get a better understanding about the general distribution of the bus stops around dublin and have the ability to drill down into specific areas.

Note: The analysis we present in this part is based on the full data records.

In [54]:
Image(url="https://drive.google.com/uc?id=1sLfArOVX4YfaQ3TuxplPUesd4h2nlttZ",  width=1500)

In the dashboard above, we can see bubbles representing the number of busStops by different partitioning of the Dublin area. The darker the color and the larger the bubble, the higher the number of bus stops in the specific areaId.

From this dashboard, vizualisations based on 'areaId1', which represents the coarsest granularity level, and the one based on  'areaId2', which represents the second coarsest granularity level (top left and top right accordingly) provide the clearest information.

From them we can see that most bus stops are in dublin's main city area, but there are also many bus stops scattered in different areas outside the main city area.

It is very difficult to derive meaningful insights from the maps corresponding to AreaId3 and AreaId (bottom left and bottom right), as at this level the bubbles are one on top of the other.
The power of these two maps appear when we zoom in into a specific part of the map, for example Dublin's Center, as we show in the dashboard below.

In [56]:
Image(url="https://drive.google.com/uc?id=1M1rrY3fACqonOc-8TN2lGCy5wMB1cRz2",  width=1500)

These two levels of fine granularity provide us with insights as to how many bus stops there are in very fine resolution, at roughly a block / street level. 

Decision makers can make use of these maps for example to better see areas with many bus stops - if they need repair, or areas without many bus stops -  if they want to expand the public transport system.

Note: computing the number of bus stops per areaId1/2/3 was not as straightforward as we thought it would be. Solving this issue required us to both perform EDA and handle uncertainty!
* Our first attempt was to simply groupby the areaId and then count distinct number of bus stops. However, following a detailed examination of the data we found out that there are records with the same bus stop but with different areaIds, even when we looked only at records with 'atStop' equals true. This lead to very skewed results the more we drilled down into the areaIds, as some bus stops were counted in 10 different AreaIds!. 
* We attribute this possibly to the way the area and bus stop is computed - through a centralized system, based on the transmitted location of the bus. And as we know that the gps sensor is not 100% accurate, this can lead to the system making this kind of mistakes.
* To handle this issue, we matched each bus stop with the most frequent areaId that it appeared with.

Note: the location on the map of each bubble is based on the centroid of each areaId. This centroid is computed based on the records that are with atStop equals true, for a better representation of the relation between the areaId and the bus stops.

## Bus Trips' lengths Analysis

Recall the bus trip definition we present in Part 1 & Part 2 Overview.

In this part, we present a granularity level analysis of the average **congestion ratio**, by granularity levels of Bus Trips' length, where the length of a bus trip is the TotalDistance (computed by summing over the distance covered along the trip).

To do so we define the following 3 granularity levels considering Bus Trips' length from the coarsest granularity level up to the finest granularity level:
1. **Very short vs not very short distance trips**: shorter than 10km vs longer than 10km.
1. **Further partition of trip distances** into very short (less than 10km), short(10-15km), short medium (15-20km), medium long (20-25km), long (25-30km) and very long (30km and above).
1. **1km resolution** for very fine analysis.

Additionally, we show an analysis of the average congestion ratio in general.


We define 'congestion ratio' of a trip as the number of records with congestion divided by total number of records in a trip.
And 'average congestion ratio' as the average across bus trips.


We believe that this analysis provides information about the general traffic and how it is connected to the length of the bus trips.

For example, maybe parts of long bus trips are out of the city and so might include less congestion during the trip, compared to short bus trips that are only around busy places.

We choose to look at TotalDistance as we think it defines a clear connection between the TotalDistance of the bus trips and the congestion percentage. This is as the TotalDistance is generally fixed given a bus trip, and thus the congestion percentage (explained variable) might be effected by the TotalDistance (explanatory variable).


**Reasoning behind granularity levels**:

* We choose 10km to be the threshold for very short distance trips. This corresponds to roughly the diameter of Dublin's main city area. 
Additionally according to [dublinpublictransport](https://www.dublinpublictransport.ie/), recently, some *short-distance* routes (previously operated by Dublin Bus) have been privatised and are provided by a company called [Go-Ahead](https://www.goaheadireland.ie/). So, we looked at the distances of bus trips that this company and see that distances are usually shorter than 10k (for example line from Tyrrelstown to Ladyswell Rd is 1.8km, and line from Bray Station to Enniskerry is 8.2km). From this inspection we determine 10km as the threshold. City and route planners can fine-tune this parameter based on their definitions.
* The partitioning into {short, medium short, ..., very long} is based on that the distance of bus trips is distributed roughly normal with an average of 20km. From this and from the def. of 'very short' bus trips we derive the 5km bucket range, and the actual buckets. 

Notes:

* the trips we refer to as 'long' are only 'long' on a metroplean level. At a state or nation wide level this distances can still be considered short. Thus this anaylsis is suitable to decision makers at the dublin city level (for example). For decision makers interested in higher-level analysis, data from dublin buses can be combined with state-wide buses, and the scales can be adjusted accordingly.
* Similarly to the outlier detection method we used in task 2, we filter out bus trips with recordCount < 20, totaldistance > 70 and uncertainPercentage > 0.01.
* The analysis we present in this part is based on the aggregated data grouped into bus trips.

In [59]:
Image(url="https://drive.google.com/uc?id=1w2xJaiZryi70Rfkw6bF1SXPAcxS16X0S",  width=1500)

In the dashboard above, we can see histograms of congestion ratio by different bus trips' lengths granularity levels.

Interesting insights:
1. In general, the average congestion ratio of a trip is 0.009, corresponding roughly 1%.  
1. We can see that very short trips (less than 10km) have considerably higher avg. congestion ratio than longer trips (more than 10km)  - 0.008 compared to 0.014, which is a 75% increase. This matches our hypothesis that very short trips are mostly in busy areas, in which it is reasonable that there will be more congestion.
1. When looking at short-long distance trips we see a slight decrease in congestion ratio.
1. Looking at the general trend, we see a sharp decline in congestion ratio till trips of length 5km, then we see the same general trend of longer distance --> lower congestion ratio, however it very quickly flattens to an almost flat / slightly downward trending line.